In [53]:
import os
import gzip
from bs4 import BeautifulSoup
import spacy_dbpedia_spotlight
import json
from collections import Counter
import pandas as pd
import numpy as np
import nltk
import spacy
import requests

In [54]:
PATH = "C:/Users/User/Downloads/project_data/project_data/cab_html_noboiler"

In [38]:
nlp = spacy.load('en_core_web_lg', disable=['ner'])
nlp.remove_pipe('tagger')
nlp.remove_pipe('parser')
nlp.add_pipe('dbpedia_spotlight',last=True)
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
def text_pipeline(text):
    tokens = []
    doc = nlp(text)
    for t in doc:
        if not t.is_stop and not t.is_punct and not t.is_space:
            tokens.append(t.lemma_.lower())
    return tokens

In [56]:
def getAbstract(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    abstract = soup.find('p', class_="lead").getText()
    return abstract

In [84]:
def entityTagger(text):
    doc=nlp(text)
    abstracts=""
    for ent in doc.ents:
        URL = ent._.dbpedia_raw_result['@URI']
        if URL not in list(ENTITIES.keys()):
            abstract=getAbstract(URL)
            abstracts+=abstract
            ENTITIES[URL]=abstract
        else:
            abstracts+=ENTITIES[URL]
    return abstracts

In [ ]:
ENTITIES={}
FILES={}

In [88]:
for root, dirs, files in os.walk(PATH, topdown=False):
    for name in files:
        if ".DS_Store" not in name:
            DIR=os.path.join(root, name).replace("\\",'/')
            if DIR not in list(FILES.keys()):
                with gzip.open(DIR, 'rb') as f:
                    lines=f.read()
                soup = BeautifulSoup(lines)
                text = soup.get_text().lower()
                FILES[DIR]=entityTagger(text)
    print("FILE DONE")

FILE DONE
FILE DONE
FILE DONE
FILE DONE
FILE DONE


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
print(FILES)